# Introduction


<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!


### Install Package with Country Codes


In [ ]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.


In [ ]:
%pip install --upgrade plotly

### Import Statements


In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation


In [4]:
pd.options.display.float_format = "{:,.2f}".format

### Load the Data


In [5]:
df_data = pd.read_csv("mission_launches_Feb2024.csv")

# Preliminary Data Exploration

- What is the shape of `df_data`?
- How many rows and columns does it have?
- What are the column names?
- Are there any NaN values or duplicates?


In [ ]:
print(df_data.shape)
df_data.tail()

In [ ]:
df_data.info()

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data.


In [8]:
df_data.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
df_data.head()

## Descriptive Statistics


In [ ]:
df_data.describe()

In [ ]:
df_data.Location.describe()

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.


In [ ]:
plt.figure(figsize=(12, 8))
sns.countplot(
    y="Organisation", data=df_data, order=df_data["Organisation"].value_counts().index
)
plt.title("Number of Space Mission Launches by Organization")
plt.xlabel("Number of Launches")
plt.ylabel("Organization")
plt.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned?


In [13]:
avr = df_data.sort_values(by="Status", ascending=False).value_counts("Status")

In [ ]:
avr

# Distribution of Mission Status

How many missions were successful?
How many missions failed?


In [ ]:
successful_missions = df_data["Mission_status"].value_counts().get("Success", 0)
print(f"Number of successful missions: {successful_missions}")

In [ ]:
failed_missions = df_data["Mission_status"].value_counts().get("Failure", 0)
print(f"Number of failed missions: {failed_missions}")

# How Expensive are the Launches?

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values).


In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(df_data["Price"].dropna(), bins=30, kde=True)
plt.title("Distribution of Launch Prices (in USD millions)")
plt.xlabel("Price (in USD millions)")
plt.ylabel(ylabel="Frequency")
plt.show()

# Use a Choropleth Map to Show the Number of Launches by Country

- Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
- Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map.
- You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

- Russia is the Russian Federation
- New Mexico should be USA
- Yellow Sea refers to China
- Shahrud Missile Test Site should be Iran
- Pacific Missile Range Facility should be USA
- Barents Sea should be Russian Federation
- Gran Canaria should be USA

You can use the iso3166 package to convert the country names to Alpha3 format.


In [ ]:
# Seperate the country from the location and create a new column.
df_data["launch_country"] = df_data["Location"].str.split(", ").str[-1]

# Replace bad data i.e outdated country names.
df_data["launch_country"] = df_data["launch_country"].replace(
    {
        "Gran Canaria": "USA",
        "Yellow Sea": "China",
        "China Coastal Waters": "China",
        "Pacific Missile Range Facility": "USA",
        "Barents Sea": "Russian Federation",
        "Russia": "Russian Federation",
        "Pacific Ocean": "USA",
        "Marshall Islands": "USA",
        "Iran": "Iran, Islamic Republic of",
        "North Korea": "Korea, Democratic People's Republic of",
        "South Korea": "Korea, Republic of",
        "United Kingdom": "United Kingdom of Great Britain and Northern Ireland",
    }
)

# Use iso3166 to get alpha3 codes
df_data["launch_country_code"] = df_data["launch_country"].apply(
    lambda x: (countries.get(x).alpha3)
)

# Use groupby to get the total launches by country and country code.
df_launches = df_data.groupby(
    ["launch_country", "launch_country_code"], as_index=False
).agg({"Mission_status": pd.Series.count})
df_launches.rename(columns={"Mission_status": "Total_launches"}, inplace=True)

# Check the table.
df_launches.sort_values(by="Total_launches", ascending=False)

In [ ]:
# Deploy the map
px.choropleth(
    data_frame=df_launches,
    locations="launch_country_code",
    color="Total_launches",
    color_continuous_scale="matter",
)

# Use a Choropleth Map to Show the Number of Failures by Country


In [ ]:
# Filter the dataframe to include only failed missions
df_failures = df_data[df_data["Mission_status"] == "Failure"]

# Group by country and count the number of failures
df_failures = df_failures.groupby(
    ["launch_country", "launch_country_code"], as_index=False
).agg({"Mission_status": pd.Series.count})
df_failures.rename(columns={"Mission_status": "Number of Failures"}, inplace=True)

df_failures.sort_values(by="Number of Failures", ascending=False)

In [ ]:
# Deploy the map for mission failures
fig = px.choropleth(
    data_frame=df_failures,
    locations="launch_country_code",
    color="Number of Failures",
    color_continuous_scale="matter",
    labels={"Number of Failures": "Number of Failures"},
    title="Number of Mission Failures by Country",
)

fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status.


In [ ]:
df_data.head()

In [ ]:
fig = px.sunburst(df_data, path=["launch_country", "Organisation", "Mission_status"])
fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions


In [ ]:
# Group by Organisation and sum the Price column
total_spent_by_org = df_data.groupby("Organisation")["Price"].sum().reset_index()

# Sort the values by the total amount spent in descending order
total_spent_by_org = total_spent_by_org.sort_values(by="Price", ascending=False)

# Display the result
total_spent_by_org.head(10)

In [ ]:
# Create a bar chart for the total amount spent by organisation
plt.figure(figsize=(12, 8))
sns.barplot(
    x="Price",
    y="Organisation",
    data=total_spent_by_org.head(20),
    hue="Organisation",
    palette="viridis",
    legend=False,
)
plt.title("Total Amount Spent by Organisation on Space Missions (in USD millions)")
plt.xlabel("Total Amount Spent (in USD millions)")
plt.ylabel("Organisation")
plt.show()

# Analyse the Amount of Money Spent by Organisation per Launch


In [ ]:
# Calculate the average price per launch for each organization
avg_spent_per_launch = df_data.groupby("Organisation")["Price"].mean().reset_index()

# Sort the values by the average amount spent per launch in descending order
avg_spent_per_launch = avg_spent_per_launch.sort_values(by="Price", ascending=False)

# Display the result
avg_spent_per_launch.head(15)

In [ ]:
# Create a bar chart for the average amount spent per launch by organization
plt.figure(figsize=(12, 8))
sns.barplot(
    x="Price",
    y="Organisation",
    data=avg_spent_per_launch.head(20),
    hue="Organisation",
    palette="viridis",
    legend=False,
)
plt.title("Average Amount Spent by Organisation per Launch (in USD millions)")
plt.xlabel("Average Amount Spent (in USD millions)")
plt.ylabel("Organisation")
plt.show()

# Chart the Number of Launches per Year


In [ ]:
# Convert the 'Datetime' column to datetime format
df_data["Datetime"] = pd.to_datetime(df_data["Datetime"], utc=True)

# Extract the year from the 'Datetime' column and create a new column 'Year'
df_data["Year"] = df_data["Datetime"].dt.year

# Group by 'Year' and count the number of launches
launches_per_year = (
    df_data.groupby("Year").size().reset_index(name="Number of Launches")
)
launches_per_year.sample(10)

In [ ]:
# Plot the number of launches per year
plt.figure(figsize=(16, 8))
sns.lineplot(data=launches_per_year, x="Year", y="Number of Launches", marker="o")
plt.title("Number of Launches per Year")
plt.xlabel("Year")
plt.ylabel("Number of Launches")
plt.grid(True)
plt.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart.


In [ ]:
# Extract the month and year from the 'Datetime' column and create a new column 'YearMonth'
df_data["YearMonth"] = df_data["Datetime"].dt.to_period("M")

# Group by 'YearMonth' and count the number of launches
launches_per_month = (
    df_data.groupby("YearMonth").size().reset_index(name="Number of Launches")
)

# Convert 'YearMonth' back to datetime for plotting
launches_per_month["YearMonth"] = launches_per_month["YearMonth"].dt.to_timestamp()

# Calculate the rolling average (e.g., 12-month rolling average)
launches_per_month["Rolling Average"] = (
    launches_per_month["Number of Launches"].rolling(window=12).mean()
)

In [ ]:
# Plot the number of launches per month with a rolling average
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=launches_per_month,
    x="YearMonth",
    y="Number of Launches",
    marker="o",
    label="Monthly Launches",
)
sns.lineplot(
    data=launches_per_month,
    x="YearMonth",
    y="Rolling Average",
    marker="o",
    label="12-Month Rolling Average",
    color="red",
)
plt.title("Number of Launches Month-on-Month with Rolling Average")
plt.xlabel("Year-Month")
plt.ylabel("Number of Launches")
plt.legend()
plt.grid(True)
plt.show()

# Find the month with the highest number of launches
max_launches_month = launches_per_month.loc[
    launches_per_month["Number of Launches"].idxmax()
]
print(
    f"The month with the highest number of launches is {max_launches_month['YearMonth'].strftime('%Y-%m')} with {max_launches_month['Number of Launches']} launches."
)

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?


In [ ]:
launches_per_month.describe()

In [ ]:
# Extract the month from the 'Datetime' column and create a new column 'Month'

df_data["Month"] = df_data["Datetime"].dt.month

# Group by 'Month' and count the number of launches

launches_per_month = df_data.groupby("Month").size().reset_index(name="Number of Launches")

# Sort the values by the number of launches in descending order

launches_per_month = launches_per_month.sort_values(by="Number of Launches", ascending=False)

# Display the result

launches_per_month


# How has the Launch Price varied Over Time?

Create a line chart that shows the average price of rocket launches over time.


In [ ]:
# Group by 'Year' and calculate the average price per year

avg_price_per_year = df_data.groupby("Year")["Price"].mean().reset_index()
avg_price_per_year.tail(10)


In [ ]:
# Plot the average price per year
plt.figure(figsize=(16, 8))
sns.lineplot(data=avg_price_per_year, x="Year", y="Price", marker="o")
plt.title("Average Price of Rocket Launches Over Time")
plt.xlabel("Year")
plt.ylabel("Average Price (in USD millions)")
plt.grid(True)
plt.show()

# Chart the Number of Launches over Time by the Top 10 Organisations.

How has the dominance of launches changed over time between the different players?


In [ ]:
# Identify the top 10 organizations by the total number of launches
top_10_orgs = df_data["Organisation"].value_counts().head(10).index

# Filter the dataframe to include only the top 10 organizations
df_top_10_orgs = df_data[df_data["Organisation"].isin(top_10_orgs)]

# Group by 'Year' and 'Organisation' and count the number of launches
launches_per_year_org = (
    df_top_10_orgs.groupby(["Year", "Organisation"])
    .size()
    .reset_index(name="Number of Launches")
)
launches_per_year_org.sort_values(by="Number of Launches", ascending=False)

In [ ]:
# Plot the number of launches over time by the top 10 organizations
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=launches_per_year_org,
    x="Year",
    y="Number of Launches",
    hue="Organisation",
    marker="o",
)
plt.title("Number of Launches over Time by the Top 10 Organisations")
plt.xlabel("Year")
plt.ylabel("Number of Launches")
plt.legend(title="Organisation")
plt.grid(True)
plt.show()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991.


In [67]:
# Filter the dataframe to include only the years up to 1991
df_cold_war = df_data[df_data["Year"] <= 1991]

# Filter the dataframe to include only the USA and USSR (including Kazakhstan as part of USSR)
df_cold_war = df_cold_war[
    df_cold_war["launch_country"].isin(["USA", "Russian Federation", "Kazakhstan"])
]

# Group by 'Year' and 'launch_country' and count the number of launches
launches_cold_war = (
    df_cold_war.groupby(["Year", "launch_country"])
    .size()
    .reset_index(name="Number of Launches")
)

In [ ]:
# Plot the number of launches over time by the USA and USSR
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=launches_cold_war,
    x="Year",
    y="Number of Launches",
    hue="launch_country",
    marker="o",
)
plt.title("Number of Launches by USA and USSR during the Cold War (up to 1991)")
plt.xlabel("Year")
plt.ylabel("Number of Launches")
plt.legend(title="Country")
plt.grid(True)
plt.show()

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches.


In [69]:
# Filter the dataframe to include only the USA and USSR (including Kazakhstan as part of USSR)
df_ussr_usa = df_data[
    df_data["launch_country"].isin(["USA", "Russian Federation", "Kazakhstan"])
]

# Group by 'launch_country' and count the number of launches
launches_ussr_usa = df_ussr_usa["launch_country"].value_counts().reset_index()
launches_ussr_usa.columns = ["launch_country", "Number of Launches"]

# Combine the counts for Russian Federation and Kazakhstan as USSR
launches_ussr_usa.loc[
    launches_ussr_usa["launch_country"].isin(["Russian Federation", "Kazakhstan"]),
    "launch_country",
] = "USSR"
launches_ussr_usa = launches_ussr_usa.groupby("launch_country").sum().reset_index()

In [ ]:
# Create the pie chart
fig = px.pie(
    launches_ussr_usa,
    names="launch_country",
    values="Number of Launches",
    title="Total Number of Launches: USSR vs USA",
    color_discrete_sequence=px.colors.sequential.Rainbow,
)

fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers


In [83]:
# Filter the dataframe to include only the USA and USSR (including Kazakhstan as part of USSR)
df_superpowers = df_data[
    df_data["launch_country"].isin(["USA", "Russian Federation", "Kazakhstan"])
]

# Group by 'Year' and 'launch_country' and count the number of launches
launches_superpowers = (
    df_superpowers.groupby(["Year", "launch_country"])
    .size()
    .reset_index(name="Number of Launches")
)

# Combine the counts for Russian Federation and Kazakhstan as USSR
launches_superpowers.loc[
    launches_superpowers["launch_country"].isin(["Russian Federation", "Kazakhstan"]),
    "launch_country",
] = "USSR"
launches_superpowers = (
    launches_superpowers.groupby(["Year", "launch_country"]).sum().reset_index()
)

In [ ]:
# Plot the number of launches year-on-year by the two superpowers
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=launches_superpowers,
    x="Year",
    y="Number of Launches",
    hue="launch_country",
    marker="o",
)
plt.title("Total Number of Launches Year-On-Year by the Two Superpowers")
plt.xlabel("Year")
plt.ylabel("Number of Launches")
plt.legend(title="Country")
plt.grid(True)
plt.show()

## Chart the Total Number of Mission Failures Year on Year.


In [85]:
# Filter the dataframe to include only failed missions
df_failures_yearly = df_data[df_data["Mission_status"] == "Failure"]

# Group by 'Year' and count the number of failures
failures_per_year = (
    df_failures_yearly.groupby("Year").size().reset_index(name="Number of Failures")
)

In [ ]:
# Plot the number of mission failures per year
plt.figure(figsize=(16, 8))
sns.lineplot(data=failures_per_year, x="Year", y="Number of Failures", marker="o")
plt.title("Total Number of Mission Failures Year on Year")
plt.xlabel("Year")
plt.ylabel("Number of Failures")
plt.grid(True)
plt.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time?


In [87]:
# Calculate the total number of launches per year
total_launches_per_year = (
    df_data.groupby("Year").size().reset_index(name="Total Launches")
)

# Calculate the number of failures per year
failures_per_year = (
    df_data[df_data["Mission_status"] == "Failure"]
    .groupby("Year")
    .size()
    .reset_index(name="Number of Failures")
)

# Merge the two dataframes on 'Year'
failures_percentage_per_year = pd.merge(
    total_launches_per_year, failures_per_year, on="Year", how="left"
)

# Calculate the percentage of failures
failures_percentage_per_year["Failure Percentage"] = (
    failures_percentage_per_year["Number of Failures"]
    / failures_percentage_per_year["Total Launches"]
) * 100

In [ ]:
# Plot the percentage of failures over time
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=failures_percentage_per_year, x="Year", y="Failure Percentage", marker="o"
)
plt.title("Percentage of Mission Failures Over Time")
plt.xlabel("Year")
plt.ylabel("Failure Percentage")
plt.grid(True)
plt.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches?


In [ ]:
# Group by 'Year' and 'launch_country' and count the number of launches
total_launches_per_year_country = (
    df_data.groupby(["Year", "launch_country"])
    .size()
    .reset_index(name="Total Launches")
)

# Find the country with the maximum launches each year
leading_country_per_year = total_launches_per_year_country.loc[
    total_launches_per_year_country.groupby("Year")["Total Launches"].idxmax()
]

# Display the result
print("Leading country per year based on total launches:")
print(leading_country_per_year)

# Filter the dataframe to include only successful missions
df_successful = df_data[df_data["Mission_status"] == "Success"]

# Group by 'Year' and 'launch_country' and count the number of successful launches
successful_launches_per_year_country = (
    df_successful.groupby(["Year", "launch_country"])
    .size()
    .reset_index(name="Successful Launches")
)

# Find the country with the maximum successful launches each year
leading_country_per_year_successful = successful_launches_per_year_country.loc[
    successful_launches_per_year_country.groupby("Year")["Successful Launches"].idxmax()
]

# Display the result
print("\nLeading country per year based on successful launches:")
print(leading_country_per_year_successful)

In [ ]:
# Plot the leading country per year based on total launches
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=leading_country_per_year,
    x="Year",
    y="Total Launches",
    hue="launch_country",
    marker="o",
)
plt.title("Leading Country per Year Based on Total Launches")
plt.xlabel("Year")
plt.ylabel("Total Launches")
plt.legend(title="Country")
plt.grid(True)
plt.show()

In [ ]:
# Plot the leading country per year based on successful launches
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=leading_country_per_year_successful,
    x="Year",
    y="Successful Launches",
    hue="launch_country",
    marker="o",
)
plt.title("Leading Country per Year Based on Successful Launches")
plt.xlabel("Year")
plt.ylabel("Successful Launches")
plt.legend(title="Country")
plt.grid(True)
plt.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020?


In [93]:
# Group by 'Year' and 'Organisation' and count the number of launches
launches_per_year_org = (
    df_data.groupby(["Year", "Organisation"])
    .size()
    .reset_index(name="Number of Launches")
)

# Find the organisation with the maximum launches each year
leading_org_per_year = launches_per_year_org.loc[
    launches_per_year_org.groupby("Year")["Number of Launches"].idxmax()
]

In [ ]:
# Plot the leading organisation per year based on total launches
plt.figure(figsize=(16, 8))
sns.lineplot(
    data=leading_org_per_year,
    x="Year",
    y="Number of Launches",
    hue="Organisation",
    marker="o",
)
plt.title("Leading Organisation per Year Based on Total Launches")
plt.xlabel("Year")
plt.ylabel("Number of Launches")
plt.legend(title="Organisation")
plt.grid(True)
plt.show()

In [ ]:
# Display the leading organisations in the 1970s, 1980s, 2018, 2019, and 2020
dominant_orgs = leading_org_per_year[
    leading_org_per_year["Year"].isin(range(1970, 1990))
    | leading_org_per_year["Year"].isin([2018, 2019, 2020])
]
print("Dominant organisations in the 1970s, 1980s, 2018, 2019, and 2020:")
print(dominant_orgs)